<a href="https://colab.research.google.com/github/madhav2381/Document-summarization-and-key-inforamtion-extraction/blob/main/CS521_Doc_Summarization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#To upload files from google drive
from google.colab import drive
drive.mount('/content/drive',force_remount=True)

Mounted at /content/drive


# pip install required packages

In [2]:
!pip install transformers[sentencepiece]
!pip install pymupdf
!pip install datasets
!pip install simplet5 -q
!pip install rouge_score

     |████████████████████████████████| 4.0 MB 5.1 MB/s 
     |████████████████████████████████| 77 kB 6.4 MB/s 
     |████████████████████████████████| 895 kB 53.3 MB/s 
     |████████████████████████████████| 596 kB 51.8 MB/s 
     |████████████████████████████████| 6.6 MB 46.1 MB/s 
     |████████████████████████████████| 1.2 MB 53.5 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
     |████████████████████████████████| 8.8 MB 5.3 MB/s 
     |████████████████████████████████| 325 kB 5.0 MB/s 
     |████████████████████████████████| 212 kB 56.9 MB/s 
     |████████████████████████████████| 1.1 MB 40.8 MB/s 
     |████████████████████████████████| 136 kB 49.7 MB/s 
     |████████████████████████████████| 127 kB 60.5 MB/s 
     |████████████████████████████████| 94 kB 3.2 MB/s 
     |████████████████████████████████| 271 kB 49.5 MB/s 
     |████████████████████████████████| 144 kB

# Import dependencies

In [3]:
import fitz
import spacy 
import re, math
import pandas as pd
from collections import Counter
from datasets import load_metric
ner = spacy.load("en_core_web_sm")

# Load Dataset

In [ ]:
from datasets import load_dataset
dataset = load_dataset("xsum")

Using custom data configuration default


Generating train split:   0%|          | 0/204045 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/11332 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/11334 [00:00<?, ? examples/s]

Dataset xsum downloaded and prepared to /root/.cache/huggingface/datasets/xsum/default/1.2.0/32c23220eadddb1149b16ed2e9430a05293768cfffbdfd151058697d4c11f934. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
# Train on 1k data records 
train_texts, train_labels = dataset['train']['document'][:1000], dataset['train']['summary'][:1000]
test_texts, test_labels = dataset['test']['document'][:50], dataset['test']['summary'][:50]

# Some models require the data with the task specification. 
# Add prefix of the task type to every data record
for i,each in enumerate(train_texts):
    train_texts[i] = 'summarize: ' + each

for i,each in enumerate(test_texts):
    test_texts[i] = 'summarize: ' + each
  
d = {'source_text':train_texts, 'target_text':train_labels}
d1 = {'source_text':test_texts, 'target_text':test_labels}
train_df = pd.DataFrame(d)
test_df = pd.DataFrame(d1)

# Models

## 1.t5

### t5-small pre-trained *model* 

In [ ]:
# consider a document from test dataset to summarize and evaluate the rouge score
test_text1, test_label1 = dataset['test']['document'][51:52], dataset['test']['summary'][51:52]

In [ ]:
test_text1[0]

'Dan Coats also told a Senate panel he did not think it was "appropriate" to discuss his conversations with the president at a public hearing.\nHis comments follow a report that President Donald Trump asked him to derail the Russia investigation.\nThe National Security Agency chief also declined to comment on the matter.\nAdmiral Mike Rogers told a Senate intelligence committee on Wednesday he has never "been directed to do anything illegal, immoral, unethical or inappropriate" as NSA director.\nMr Coats echoed Mr Rogers\' statements as senators pressed the pair on their interactions with the president.\n"I\'m willing to come before the committee and tell you what I know and don\'t know," he said. "What I\'m not willing to do is share information I think ought to be protected in an opening hearing," Mr Coats told the panel.\nHe is later appearing in a closed session before the committee on Wednesday afternoon.\nThis Senate testimony by intelligence community heads was billed as the und

In [ ]:
test_label1[0]

'The US director of national intelligence has said he "never felt pressured" to influence the inquiry into Russia\'s political meddling.'

In [ ]:
# t5-small pre-trained model 
from transformers import T5Tokenizer, T5ForConditionalGeneration

tokenizer = T5Tokenizer.from_pretrained("t5-small")
model = T5ForConditionalGeneration.from_pretrained("t5-small")

input_ids = tokenizer(test_text1[0], max_length=1024, truncation=True, return_tensors="pt").input_ids
labels = tokenizer(test_label1[0],  max_length=1024, truncation=True, return_tensors="pt").input_ids
# the forward function automatically creates the correct decoder_input_ids
loss = model(input_ids=input_ids, labels=labels).loss

outputs = model.generate(input_ids)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))
sum_t5_pre = tokenizer.decode(outputs[0], skip_special_tokens=True)

In [ ]:
sum_t5_pre

'a dud, but he will have more leeway to discuss his interactions'

In [ ]:
# Rouge Scores
metric = load_metric("rouge")
def calc_rouge_scores(candidates, references):
    result = metric.compute(predictions=candidates, references=references, use_stemmer=True)
    result = {key: round(value.mid.fmeasure * 100, 1) for key, value in result.items()}
    return result
  
calc_rouge_scores([sum_t5_pre], test_label1)

{'rouge1': 12.1, 'rouge2': 0.0, 'rougeL': 12.1, 'rougeLsum': 12.1}

In [ ]:
WORD = re.compile(r'\w+')

def get_cosine(vec1, vec2):
     intersection = set(vec1.keys()) & set(vec2.keys())
     numerator = sum([vec1[x] * vec2[x] for x in intersection])

     sum1 = sum([vec1[x]**2 for x in vec1.keys()])
     sum2 = sum([vec2[x]**2 for x in vec2.keys()])
     denominator = math.sqrt(sum1) * math.sqrt(sum2)

     if not denominator:
        return 0.0
     else:
        return float(numerator) / denominator

def text_to_vector(text):
     words = WORD.findall(text)
     return Counter(words)

In [ ]:
# Cosine similarity
vector1 = text_to_vector(test_label1[0])
vector2 = text_to_vector(sum_t5_pre)

cosine = get_cosine(vector1, vector2)

print ('Cosine:', cosine)

Cosine: 0.12598815766974242


### **Fine-tune T5**

In [ ]:
from simplet5 import SimpleT5

model = SimpleT5()
model.from_pretrained(model_type="t5", model_name="t5-base")

Global seed set to 42


Downloading:   0%|          | 0.00/773k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.32M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.17k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/850M [00:00<?, ?B/s]

In [ ]:
# Fine tune with the xsum dataset 

model.train(train_df=train_df,
            eval_df = test_df,
            source_max_token_len=128, 
            target_max_token_len=50, 
            batch_size=8, max_epochs=5, use_gpu=True)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
Missing logger folder: /content/lightning_logs

  | Name  | Type                       | Params
-----------------------------------------------------
0 | model | T5ForConditionalGeneration | 222 M 
-----------------------------------------------------
222 M     Trainable params
0         Non-trainable params
222 M     Total params
891.614   Total estimated model params size (MB)


Validation sanity check: 0it [00:00, ?it/s]

Global seed set to 42


Training: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

In [ ]:
! ( cd outputs; ls )

simplet5-epoch-0-train-loss-2.638-val-loss-2.445
simplet5-epoch-1-train-loss-2.2703-val-loss-2.4175
simplet5-epoch-2-train-loss-2.0365-val-loss-2.4662
simplet5-epoch-3-train-loss-1.8212-val-loss-2.5292
simplet5-epoch-4-train-loss-1.6222-val-loss-2.6168


In [ ]:
model.load_model('t5', 'outputs/simplet5-epoch-4-train-loss-1.6222-val-loss-2.6168', use_gpu=True)

In [ ]:
test_text1, test_label1 = dataset['test']['document'][51:52], dataset['test']['summary'][51:52]


In [ ]:
candidates = model.predict(test_text1[0])

In [ ]:
candidates

['The former NSA director Dan Coats has told a Senate panel he does not think it is appropriate to discuss his interactions with the president at an open hearing.']

In [ ]:
# Rouge scores
calc_rouge_scores(candidates, test_label1)

{'rouge1': 24.0, 'rouge2': 0.0, 'rougeL': 24.0, 'rougeLsum': 24.0}

In [ ]:
# Cosine similarity
vector1 = text_to_vector(test_label1[0])
vector2 = text_to_vector(candidates[0])

cosine = get_cosine(vector1, vector2)

print ('Cosine:', cosine)

Cosine: 0.24313226954193234


## 2.Pegasus

### Pegasus pre-trained model

In [ ]:
from transformers import PegasusForConditionalGeneration, PegasusTokenizer
import torch

model_name = "google/pegasus-large"
tokenizer = PegasusTokenizer.from_pretrained(model_name)
model = PegasusForConditionalGeneration.from_pretrained(model_name)
batch = tokenizer(test_texts[51:52], truncation=True, padding="longest", return_tensors="pt")
translated = model.generate(**batch)
sum_pegasus_pre = tokenizer.batch_decode(translated, skip_special_tokens=True)

In [ ]:
sum_pegasus_pre

In [ ]:
calc_rouge_scores([sum_pegasus_pre], test_label1)

In [ ]:
# Cosine similarity
vector1 = text_to_vector(test_label1[0])
vector2 = text_to_vector(sum_pegasus_pre)

cosine = get_cosine(vector1, vector2)

print ('Cosine:', cosine)

### Fine-tune Pegasus 

In [ ]:
from transformers import PegasusForConditionalGeneration, PegasusTokenizer, Trainer, TrainingArguments
import torch

class PegasusDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels
    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels['input_ids'][idx])  # torch.tensor(self.labels[idx])
        return item
    def __len__(self):
        return len(self.labels['input_ids'])  # len(self.labels)

      
def prepare_data(model_name, 
                 train_texts, train_labels, 
                 val_texts=None, val_labels=None, 
                 test_texts=None, test_labels=None):
  """
  Prepare input data for model fine-tuning
  """
  tokenizer = PegasusTokenizer.from_pretrained(model_name)

  prepare_val = False if val_texts is None or val_labels is None else True
  prepare_test = False if test_texts is None or test_labels is None else True

  def tokenize_data(texts, labels):
    encodings = tokenizer(texts, truncation=True, padding=True)
    decodings = tokenizer(labels, truncation=True, padding=True)
    dataset_tokenized = PegasusDataset(encodings, decodings)
    return dataset_tokenized

  train_dataset = tokenize_data(train_texts, train_labels)
  val_dataset = tokenize_data(val_texts, val_labels) if prepare_val else None
  test_dataset = tokenize_data(test_texts, test_labels) if prepare_test else None

  return train_dataset, val_dataset, test_dataset, tokenizer


def prepare_fine_tuning(model_name, tokenizer, train_dataset, val_dataset=None, freeze_encoder=False, output_dir='./results'):
  """
  Prepare configurations and base model for fine-tuning
  """
  torch_device = 'cuda' if torch.cuda.is_available() else 'cpu'
  model = PegasusForConditionalGeneration.from_pretrained(model_name) #.to(torch_device)

  if freeze_encoder:
    for param in model.model.encoder.parameters():
      param.requires_grad = False

  if val_dataset is not None:
    training_args = TrainingArguments(
      output_dir=output_dir,           # output directory
      num_train_epochs=5,           # total number of training epochs
      per_device_train_batch_size=1,   # batch size per device during training, can increase if memory allows
      per_device_eval_batch_size=1,    # batch size for evaluation, can increase if memory allows
      save_steps=500,                  # number of updates steps before checkpoint saves
      save_total_limit=5,              # limit the total amount of checkpoints and deletes the older checkpoints
      evaluation_strategy='steps',     # evaluation strategy to adopt during training
      eval_steps=100,                  # number of update steps before evaluation
      warmup_steps=500,                # number of warmup steps for learning rate scheduler
      weight_decay=0.01,               # strength of weight decay
      logging_dir='./logs',            # directory for storing logs
      logging_steps=10,
    )

    trainer = Trainer(
      model=model,                         # the instantiated 🤗 Transformers model to be trained
      args=training_args,                  # training arguments, defined above
      train_dataset=train_dataset,         # training dataset
      eval_dataset=val_dataset,            # evaluation dataset
      tokenizer=tokenizer
    )

  else:
    training_args = TrainingArguments(
      output_dir=output_dir,           # output directory
      num_train_epochs=1,           # total number of training epochs
      per_device_train_batch_size=1,   # batch size per device during training, can increase if memory allows
      save_steps=500,                  # number of updates steps before checkpoint saves
      save_total_limit=5,              # limit the total amount of checkpoints and deletes the older checkpoints
      warmup_steps=500,                # number of warmup steps for learning rate scheduler
      weight_decay=0.01,               # strength of weight decay
      logging_dir='./logs',            # directory for storing logs
      logging_steps=10,
    )

    trainer = Trainer(
      model=model,                         # the instantiated 🤗 Transformers model to be trained
      args=training_args,                  # training arguments, defined above
      train_dataset=train_dataset,         # training dataset
      tokenizer=tokenizer
    )

  return trainer


if __name__=='__main__':
  # use XSum dataset as example, with first 1000 docs as training data
  #from datasets import load_dataset
  #dataset = load_dataset("xsum")
  #train_texts, train_labels = dataset['train']['document'][:1000], dataset['train']['summary'][:1000]
  
  # use Pegasus Large model as base for fine-tuning
  model_name = 'google/pegasus-large'
  train_dataset, _, _, tokenizer = prepare_data(model_name, train_texts, train_labels)
  trainer = prepare_fine_tuning(model_name, tokenizer, train_dataset)
  trainer.train()

Using custom data configuration default


Generating train split:   0%|          | 0/204045 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/11332 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/11334 [00:00<?, ? examples/s]

Dataset xsum downloaded and prepared to /root/.cache/huggingface/datasets/xsum/default/1.2.0/32c23220eadddb1149b16ed2e9430a05293768cfffbdfd151058697d4c11f934. Subsequent calls will reuse this data.


...

Downloading:   0%|          | 0.00/88.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/3.02k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.12G [00:00<?, ?B/s]

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 1000
  Num Epochs = 1
  Instantaneous batch size per device = 1
  Total train batch size (w. parallel, distributed & accumulation) = 1
  Gradient Accumulation steps = 1
  Total optimization steps = 1000


Step,Training Loss


## 3.distilbart

### distilbart pre-trained model

In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

checkpoint = 'sshleifer/distilbart-cnn-12-6'
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = AutoModelForSeq2SeqLM.from_pretrained(checkpoint)

In [ ]:
input_ids = tokenizer(test_text1[0], max_length=1024, truncation=True, return_tensors="pt").input_ids
labels = tokenizer(test_label1[0],  max_length=1024, truncation=True, return_tensors="pt").input_ids
# the forward function automatically creates the correct decoder_input_ids
loss = model(input_ids=input_ids, labels=labels).loss

outputs = model.generate(input_ids)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))
sum_distilbart_pre = tokenizer.decode(outputs[0], skip_special_tokens=True)

In [ ]:
sum_distilbart_pre

In [ ]:
# Rouge scores
calc_rouge_scores([sum_distilbart_pre], test_label1)

In [ ]:
# Cosine similarity
vector1 = text_to_vector(test_label1[0])
vector2 = text_to_vector(sum_distilbart_pre)

cosine = get_cosine(vector1, vector2)

print ('Cosine:', cosine)

# Long documents summarization

In [ ]:
file = fitz.open('/content/Contract - Energy Matrix.pdf')

In [4]:
file2 = fitz.open('/content/SME-Agreement-in-English.pdf')

In [5]:
#Extracts the text from the pdf documents in sequential way
def extract_text(pdf) :
  text=""                          #empty string
  for i in range(pdf.page_count) : # pdf.page_count gives the total no.of pages within a pdf
    text+=pdf.get_page_text(i)     # pdf.get_page_text takes the integer value as parameter and gives the text from that (integer+1) page
  return text

In [ ]:
file_content = extract_text(file)

In [6]:
file_content2 = extract_text(file2)

In [ ]:
len(file_content)

150113

In [7]:
len(file_content2)

42345

In [8]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

checkpoint = 'sshleifer/distilbart-cnn-12-6'
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = AutoModelForSeq2SeqLM.from_pretrained(checkpoint)

Downloading:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.76k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/878k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.14G [00:00<?, ?B/s]

In [ ]:
tokenizer.model_max_length

1024

In [ ]:
tokenizer.max_len_single_sentence

1022

In [9]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
sentences = nltk.tokenize.sent_tokenize(file_content)

In [10]:
sentences2 = nltk.tokenize.sent_tokenize(file_content2)

In [ ]:
len(sentences)

812

In [11]:
len(sentences2)

186

In [ ]:
max(len(tokenizer.tokenize(sentence)) for sentence in sentences)

523

In [ ]:
# initialize for file1 - Energy matrix 
length = 0
chunk = ""
chunks = []
count = -1
for sentence in sentences:
  count += 1
  combined_length = len(tokenizer.tokenize(sentence)) + length # add the no. of sentence tokens to the length counter

  if combined_length  <= tokenizer.max_len_single_sentence: # if it doesn't exceed
    chunk += sentence + " " # add the sentence to the chunk
    length = combined_length # update the length counter

    # if it is the last sentence
    if count == len(sentences) - 1:
      chunks.append(chunk.strip()) # save the chunk
    
  else: 
    chunks.append(chunk.strip()) # save the chunk
    
    # reset 
    length = 0 
    chunk = ""

    # take care of the overflow sentence
    chunk += sentence + " "
    length = len(tokenizer.tokenize(sentence))
len(chunks)

37

In [ ]:
# inputs to the model
inputs = [tokenizer(chunk, return_tensors="pt") for chunk in chunks]

In [ ]:
summary = ''
for input in inputs: 
  output = model.generate(**input)
  print(tokenizer.decode(*output, skip_special_tokens=True))
  summary += tokenizer.decode(*output, skip_special_tokens=True)

 The loan contract, hereinafter the “Contract,” is entered into between the COOPERATIVE REPUBLIC OF GUYANA and the  INTER-AMERICAN DEVELOPMENT BANK. The purpose of this Contract is to establish the terms and conditions under which the Bank grants a loan to the Borrower to contribute to the program “Energy Matrix Diversification and Institutional Strengthening of the Department of Energy (EMISDE)”
 In accordance with this Contract, the Bank agrees to lend to the Borrower, and the Bank accepts, a loan of up to the amount of twenty-one ipientmillion one hundred and sixty thousand dollars (US$21,160,000) hereinafter the “Loan’s” The Bank agrees to lend the loan to the ‘Borrower’, ‘The Bank agrees’ to lend to  the Bank, “The Bank is unable to obtain access to Dollars,’ the Bank may disburse the Loan in another currency of its ‘wallet’
 The Borrower may request Currency Conversions and/or Interest Rate Conversions from the Bank at any time during the term of the Contract. The Concessional OC

In [ ]:
len(summary)

In [13]:
# initialize for file2 - SME-Agreement-in-English
length = 0
chunk = ""
chunks = []
count = -1
for sentence in sentences2:
  count += 1
  combined_length = len(tokenizer.tokenize(sentence)) + length # add the no. of sentence tokens to the length counter

  if combined_length  <= tokenizer.max_len_single_sentence: # if it doesn't exceed
    chunk += sentence + " " # add the sentence to the chunk
    length = combined_length # update the length counter

    # if it is the last sentence
    if count == len(sentences2) - 1:
      chunks.append(chunk.strip()) # save the chunk
    
  else: 
    chunks.append(chunk.strip()) # save the chunk
    
    # reset 
    length = 0 
    chunk = ""

    # take care of the overflow sentence
    chunk += sentence + " "
    length = len(tokenizer.tokenize(sentence))
len(chunks)

12

In [14]:
# inputs to the model
inputs = [tokenizer(chunk, return_tensors="pt") for chunk in chunks]

In [15]:
summary2 = ''
for input in inputs: 
  output = model.generate(**input)
  print(tokenizer.decode(*output, skip_special_tokens=True))
  summary2 += tokenizer.decode(*output, skip_special_tokens=True)

 The Borrower, the Guarantor(s) and the Lender shall hereinafter be referred to individually as “Parties” The Lender is engaged in carrying on bonafide business activities as set out in Schedule 1. The Lenders are engaged in providing finance to a wide range of customers including small and medium enterprises.
 The Borrower agrees to pay to the Lender the fees as set out in Schedule 2. The obligation of  the entire Loan Amount along with Interest, and other charges together with applicable taxes. The Lender may swap the PDC’s issued or request for other mode of payment towards discharge of the liability.
 The Borrower may exercise the option of pre-payment post Lock-in-period subject to the following conditions and at the absolute discretion of the Lender. The amount pre paid shall be first used to credit the overdue interest, bank charges, legal expenses if any, Arrears of instalments, interest due upto date for the current month, interest and other expenses and taxes due. If the enti

In [16]:
summary2

' The Borrower, the Guarantor(s) and the Lender shall hereinafter be referred to individually as “Parties” The Lender is engaged in carrying on bonafide business activities as set out in Schedule 1. The Lenders are engaged in providing finance to a wide range of customers including small and medium enterprises. The Borrower agrees to pay to the Lender the fees as set out in Schedule 2. The obligation of  the entire Loan Amount along with Interest, and other charges together with applicable taxes. The Lender may swap the PDC’s issued or request for other mode of payment towards discharge of the liability. The Borrower may exercise the option of pre-payment post Lock-in-period subject to the following conditions and at the absolute discretion of the Lender. The amount pre paid shall be first used to credit the overdue interest, bank charges, legal expenses if any, Arrears of instalments, interest due upto date for the current month, interest and other expenses and taxes due. If the entir

In [17]:
len(summary2)

4923

# Key information extraction

In [19]:
# creating a dictionary of lists to store the words as values and tags as keys
def ner_tags(d) :
  tags={}    #empty dictionary
  for each in d.ents :
    if each.label_ in tags : 
      tags[each.label_].append(each.text)
    else :
      tags[each.label_]=[each.text]
  return tags

In [ ]:
# NE tags for energy matrix pdf

d=ner(summary)
tags1=ner_tags(d)
for each in tags1 :
  print(each," : ",tags1[each])

WORK_OF_ART  :  ['Contract', 'Loan', 'Original Disbursement Period', 'the Special Conditions', 'Guarantor in the Guarantee Contracts']
ORG  :  ['COOPERATIVE', 'INTER-AMERICAN DEVELOPMENT BANK', 'Bank', 'Energy Matrix Diversification', 'the Department of Energy', 'Bank', 'Bank', 'Bank', 'Bank', 'Bank', 'Bank', 'Bank', 'the Additional Resources', 'the Bank of Guyana', 'Bank', 'Bank', 'POM', 'POM', 'Program', 'Program', 'Bank', 'Program', 'PEP', 'AOP', 'Tribunal', 'Bank', 'Disbursement Period', 'Concessional Ordinary Capital', 'Bank', 'The Inter-American Development Bank', 'Executing Agency', 'Financing Facility’', 'Bank', 'The LIBOR Interest Rate for an Interest Rate Determination Date', 'the Reference Banks', 'Prohibited Practices', 'Bank', 'Board', 'Board', 'Bank', 'Bank', 'Bank', 'WAL', 'the Amortized Schedules', 'WAL', 'Bank', 'Concessional', 'Bank', 'Bank', 'Bank', 'Project', 'Eligible Expenditures', 'the Executing Agency', 'Loan', 'the Additional Resources', 'Conversion Request Let

In [20]:
# NE tags for energy matrix pdf

d2=ner(summary2)
tags2=ner_tags(d2)
for each in tags2 :
  print(each," : ",tags2[each])

PERSON  :  ['Lender', 'Lender', 'Lender', 'Lock', 'Lender', 'Lender', 'Lender', 'Lender', 'Lender', 'Lender']
WORK_OF_ART  :  ['The Lender', 'Lender', 'RBI']
FAC  :  ['Schedule 1', 'Schedule 2']
ORG  :  ['Loan Amount', 'PDC', 'EMI', 'EMI', 'Borrowers and Guarantors', 'the Insurance Company / Government', 'Borker', 'the Reserve Bank of India', 'the Credit Information Companies']
ORDINAL  :  ['first', 'third', 'First']
DATE  :  ['the current month', 'month', 'month', '1996', 'one year', 'one year', '2 years']
PRODUCT  :  ['Borrower', 'Borrower']
CARDINAL  :  ['two or more', 'one', 'one']
LAW  :  ['the Arbitration Act']


# Question-answering

In [2]:
!pip install transformers[sentencepiece]

     |████████████████████████████████| 4.0 MB 7.9 MB/s 
     |████████████████████████████████| 77 kB 5.0 MB/s 
     |████████████████████████████████| 895 kB 63.7 MB/s 
     |████████████████████████████████| 6.6 MB 39.1 MB/s 
     |████████████████████████████████| 596 kB 41.9 MB/s 
     |████████████████████████████████| 1.2 MB 43.6 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [3]:
from transformers import pipeline 

qa = pipeline("question-answering")

No model was supplied, defaulted to distilbert-base-cased-distilled-squad (https://huggingface.co/distilbert-base-cased-distilled-squad)


Downloading:   0%|          | 0.00/473 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/249M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/208k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/426k [00:00<?, ?B/s]

## QA for pdf1 summary

In [ ]:
question1 = "Which organisation signs the Loan Contract?"
answer1 = qa(question=question1, context=summary) #takes the question and text(summaries) in our case as parameters

question2 = "Which organisation other than " + answer1['answer'] + " signs the Loan Contract?"
answer2 = qa(question=question2, context=summary) #takes the question and text(summaries) in our case as parameters

question3 = "What is the money taken?"
answer3 = qa(question=question3, context=summary) #takes the question and text(summaries) in our case as parameters

# Print the answer
print(f"Question: {question1}")
print(f"Answer: '{answer1['answer']}' with score {answer1['score']}") 

print(f"Question: {question2}")
print(f"Answer: '{answer2['answer']}' with score {answer2['score']}")

print(f"Question: {question3}")
print(f"Answer: '{answer3['answer']}' with score {answer3['score']}")


/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:703: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  tensor = as_tensor(value)

/usr/local/lib/python3.7/dist-packages/transformers/pipelines/question_answering.py:300: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  for span_id in range(num_spans)



Question: Which organisation signs the Loan Contract?
Answer: 'Executing Agency and Contracting Agency' with score 0.6394587755203247
Question: Which organisation other than Executing Agency and Contracting Agency signs the Loan Contract?
Answer: 'co-executing agencies' with score 0.8748259544372559
Question: What is the money taken?
Answer: 'the undisbursed amount is less' with score 0.31059131026268005


In [ ]:
question4 = "What is the interest rate?"
answer4 = qa(question=question4, context=summary);

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:703: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  tensor = as_tensor(value)

/usr/local/lib/python3.7/dist-packages/transformers/pipelines/question_answering.py:300: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  for span_id in range(num_spans)



In [ ]:
question8 = "How much amount is lent from the bank?"
answer8 = qa(question=question8, context=summary);

print(f"Question: {question8}")
print(f"Answer: '{answer8['answer']}' with score {answer8['score']}");

Question: How much amount is lent from the bank?
Answer: 'twenty-one ipientmillion one hundred and sixty thousand dollars' with score 0.2935749888420105


In [ ]:
print(f"Question: {question4}")
print(f"Answer: '{answer4['answer']}' with score {answer4['score']}")

Question: What is the interest rate?
Answer: 'LIBOR Interest Rate in Dollars' with score 0.2227422446012497


In [ ]:
question5 = "What is the name of bank?"
answer5 = qa(question=question5, context=summary);

print(f"Question: {question5}")
print(f"Answer: '{answer5['answer']}' with score {answer5['score']}");

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:703: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  tensor = as_tensor(value)

/usr/local/lib/python3.7/dist-packages/transformers/pipelines/question_answering.py:300: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  for span_id in range(num_spans)



Question: What is the name of bank?
Answer: 'Bank of Guyana' with score 0.7105961441993713


In [ ]:
question6 = "when is the loan repayment?"
answer6 = qa(question=question6, context=summary);

print(f"Question: {question6}")
print(f"Answer: '{answer6['answer']}' with score {answer6['score']}");

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:703: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  tensor = as_tensor(value)

/usr/local/lib/python3.7/dist-packages/transformers/pipelines/question_answering.py:300: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  for span_id in range(num_spans)



Question: when is the loan repayment?
Answer: 'fifteenth (15th) day of the month' with score 0.3958682715892792


## QA for pdf2 summary

In [6]:
summary2 = " The Borrower, the Guarantor(s) and the Lender shall hereinafter be referred to individually as “Parties” The Lender is engaged in carrying on bonafide business activities as set out in Schedule 1. The Lenders are engaged in providing finance to a wide range of customers including small and medium enterprises. The Borrower agrees to pay to the Lender the fees as set out in Schedule 2. The obligation of  the entire Loan Amount along with Interest, and other charges together with applicable taxes. The Lender may swap the PDC’s issued or request for other mode of payment towards discharge of the liability. The Borrower may exercise the option of pre-payment post Lock-in-period subject to the following conditions and at the absolute discretion of the Lender. The amount pre paid shall be first used to credit the overdue interest, bank charges, legal expenses if any, Arrears of instalments, interest due upto date for the current month, interest and other expenses and taxes due. If the entire amount due for prepayment henyis not remitted then the amount so received by the. Lender would be treated as an advance ipient payment of future EMI and will be. adjusted to  EMI month on month, as and when The liability of the Borrower and the Guarantor(s) shall be joint and several,  notwithstanding that any Security or Securities comprised in any instrument(s), comprised in. any instrument executed or to be executed by the. Borrowers and Guarantors (s) agree that, the Lender shall have right of ipientlien over all the assets of  the. The Lender has right to recall the entire loan together with interest for the entire period, the overdue interest arising on account of an event of default. The Borrower authorizes the Lender to receive any amount that may be paid by the Insurance Company / Government body, as against any insurance policy that may have been. The Lender is not liable for admission of any claim by the insurance company and the settlement. Lender may initiate criminal proceeding or any other appropriate appropriate  actions against the Borrowers and the Guarantor. Lender has the right to combine or consolidate or divide into two or more of any of the accounts of the Borrower  with the Lender and/or set-off or transfer or appropriate any sum or sums  in the credit of any one of such accounts in or towards part or full atisfaction of the liabilities. Lender shall be entitled to assign / securitize its receivables from Borrowers. The Borrower and Guarantor(s) shall bear all costs and other expenses incurred in the completion of this Agreement and in complying with the terms and conditions of the Agreement. The parties agree that any confirmation for further loan facility as confirmed and accepted by the Borrowers over the phone is binding. The Parties also agree that the Lender may engage one or more person(s / agencies) to verify any fact or. information furnished by, concerning and. pertaining to, concerning. and pertaining to the Borker and  “Guarantor(S)’s) and collect the outstanding amount and / or to enforce any security. The Guarantor(s) are  liable to repay the Loan Amount along with Interest in terms  of the repayment schedule. The Borrower and/or the Guarantor will not seek to claim or recover from the Lender for any purported damages or compensation, direct, indirect or  consequential, for any acts or actions whatsoever. The Lender shall be entitled to assign its rights to its obligations under this Agreement to a third party. Agreement becomes effective on execution. Arbitration and Dispute Settlement shall be settled by arbitration in accordance with the provisions of the Arbitration Act, 1996 or any other statutory modification or re-enactment for the time being in force and shall be conducted by a sole arbitrator. The Borrower and Guarantor(s) accept, confirm and consent for the disclosure and  sharing by the Lender of all or any information and data. The Lender may deem appropriate  and necessary to disclose and furnish, to the Reserve Bank of India (“RBI”) and/or to  the Credit Information Companies, to other banks and lenders including assignees and potential. The Lender may (but not obliged to) send short message services (sms) to the ipientBorrower intimating him on the dues payable by him and may call the Borrower and  or the guarantor to pay any dues that is outstanding under the agreement. The Borrowers and the Guarantor(s) have read the entire Agreement, constituting the above  grotesqueclauses including the Loan details and the terms of repayment. Pre-payment charges include pre-closing charges on the principal outstanding if the loan is pre-closed within one year. Pre-closed after one year but before 2 years. Additional charges on principal outstanding will be charged if loan is taken over by other financiers. Insurance premium for First year(optional) is optional. Credit shield (optional) for Rs.1000/£200/£1,000."

In [10]:
# for pdf2

question8 = "How much amount is lent from the bank?"
answer8 = qa(question=question8, context=summary2);

print(f"Question: {question8}")
print(f"Answer: '{answer8['answer']}' with score {answer8['score']}");

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:703: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  tensor = as_tensor(value)
/usr/local/lib/python3.7/dist-packages/transformers/pipelines/question_answering.py:300: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  for span_id in range(num_spans)


Question: How much amount is lent from the bank?
Answer: 'Rs.1000/£200/£1,000' with score 0.7282254099845886


In [12]:
question6 = "when is the loan repayment?"
answer6 = qa(question=question6, context=summary2);

print(f"Question: {question6}")
print(f"Answer: '{answer6['answer']}' with score {answer6['score']}");

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:703: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  tensor = as_tensor(value)
/usr/local/lib/python3.7/dist-packages/transformers/pipelines/question_answering.py:300: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  for span_id in range(num_spans)


Question: when is the loan repayment?
Answer: 'within one year' with score 0.3100891709327698


In [13]:
question5 = "What is the name of bank?"
answer5 = qa(question=question5, context=summary2);

print(f"Question: {question5}")
print(f"Answer: '{answer5['answer']}' with score {answer5['score']}");

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:703: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  tensor = as_tensor(value)
/usr/local/lib/python3.7/dist-packages/transformers/pipelines/question_answering.py:300: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  for span_id in range(num_spans)


Question: What is the name of bank?
Answer: 'Reserve Bank of India' with score 0.6035973429679871


In [14]:
question1 = "Which organisation signs the Loan Contract?"
answer1 = qa(question=question1, context=summary2) #takes the question and text(summaries) in our case as parameters

question2 = "Which organisation other than " + answer1['answer'] + " signs the Loan Contract?"
answer2 = qa(question=question2, context=summary2) #takes the question and text(summaries) in our case as parameters


# Print the answer
print(f"Question: {question1}")
print(f"Answer: '{answer1['answer']}' with score {answer1['score']}") 

print(f"Question: {question2}")
print(f"Answer: '{answer2['answer']}' with score {answer2['score']}")


/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:703: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  tensor = as_tensor(value)
/usr/local/lib/python3.7/dist-packages/transformers/pipelines/question_answering.py:300: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  for span_id in range(num_spans)


Question: Which organisation signs the Loan Contract?
Answer: 'The Borrowers' with score 0.5187896490097046
Question: Which organisation other than The Borrowers signs the Loan Contract?
Answer: 'Guarantors' with score 0.8233965039253235
